In [1]:
%load_ext autoreload
%autoreload 2

import chromadb
import requests



# Add cached image

In [23]:
import requests

url = "http://0.0.0.0:8017/upsert_cached_image/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "persona": 'nikolai', 
    "img_uuid": "olalatropolya",
    "img_path": "example/img/path1.jpeg",
    "prompt": "nuclear sex",
    "emb": [0.0 for i in range(1024)], # dimensions should be same in same collection
    "tags": {
        "tag_outdoors": 0,
        "tag_intimacy_lvl": 2,
        "tag_pussy": 1
    }
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

200
None


# Query image

In [24]:
### You can create function like this:

# async def query_cached_image(avatar, user_id, diffusion_prompt, tags={}, n_results=1):
    
#     if is_running_in_docker() or is_running_in_kubernetes():
#         host = 'localhost'
#     else:
#         host = 'imagecache'
        
#     url = f"http://{host}:8017/get_image_candidate/"
#     headers = {"accept": "application/json", "Content-Type": "application/json"}
#     payload = {
#         "diffusion_prompt": diffusion_prompt,
#         "persona": avatar,
#         "user_id": user_id,
#         "tags": tags,
#         "n_results": n_results
#     }

#     response = requests.post(url, headers=headers, json=payload)

#     print(response.status_code)
#     print(response.json())
    
    
    

import requests

url = "http://0.0.0.0:8017/get_image_candidate/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "diffusion_prompt": "atomic sex",
    "persona": "nikolai",
    "user_id": 3,
    "tags": {"tag_outdoors": 0},
    "n_results": 1
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

200
{"ids": [["olalatropolya"]], "distances": [[1.0000002072227896]], "embeddings": null, "metadatas": [[{"img_uuid": "olalatropolya", "intimacy_lvl": 2, "outdoors": 1, "prompt": "nuclear sex", "pussy": 1, "tag_intimacy_lvl": 2, "tag_outdoors": 0, "tag_pussy": 1}]], "documents": [["example/img/path1.jpeg"]], "uris": null, "data": null, "included": ["metadatas", "documents", "distances"]}


In [26]:
# Change tag in request
import requests

payload['tags'] = {"tag_outdoors": 1}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

200
{"ids": [[]], "distances": [[]], "embeddings": null, "metadatas": [[]], "documents": [[]], "uris": null, "data": null, "included": ["metadatas", "documents", "distances"]}


# Add record, to avoid using same image for same user_id

In [28]:
import requests

url = "http://0.0.0.0:8017/add_image_usage_record/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "user_id": 3,
    "img_uuid": "olalatropolya",
    "img_path": "string" # This is optional
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

200
None


* After adding record, it will exclude this image from using by same user id

In [29]:
import requests

url = "http://0.0.0.0:8017/get_image_candidate/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "diffusion_prompt": "atomic sex",
    "persona": "nikolai",
    "user_id": 3,
    "tags": {"tag_outdoors": 0},
    "n_results": 1
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

200
{"ids": [[]], "distances": [[]], "embeddings": null, "metadatas": [[]], "documents": [[]], "uris": null, "data": null, "included": ["metadatas", "documents", "distances"]}


# Chroma queries

In [25]:
import chromadb
chroma_client = chromadb.HttpClient(host="localhost", port=8000, settings=chromadb.Settings(allow_reset=True, anonymized_telemetry=False))
collection = chroma_client.get_collection(name='nikolai')
collection.get()

{'ids': ['olalatropolya'],
 'embeddings': None,
 'metadatas': [{'img_uuid': 'olalatropolya',
   'intimacy_lvl': 2,
   'outdoors': 1,
   'prompt': 'nuclear sex',
   'pussy': 1,
   'tag_intimacy_lvl': 2,
   'tag_outdoors': 0,
   'tag_pussy': 1}],
 'documents': ['example/img/path1.jpeg'],
 'data': None,
 'uris': None,
 'included': ['metadatas', 'documents']}